# Лабораторная работа 2
## Задание:
1. Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Подключение библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

### Загрузка данных и подготовка датасета

In [2]:
dataSet = pd.read_csv('cybersecurity_attacks.csv')

label_encoder = LabelEncoder()
dataSet['Attack Type Encoded'] = label_encoder.fit_transform(dataSet['Attack Type'])

non_numeric_columns = dataSet.select_dtypes(exclude=['number']).columns.tolist()

dataSet = dataSet.drop(columns=non_numeric_columns)
dataSet.dropna(inplace=True)
print(dataSet)

       Source Port  Destination Port  Packet Length  Anomaly Scores  \
0            31225             17616            503           28.67   
1            17245             48166           1174           51.50   
2            16811             53600            306           87.42   
3            20018             32534            385           15.79   
4             6131             26646           1462            0.52   
...            ...               ...            ...             ...   
39995        31005              6764           1428           39.28   
39996         2553             28091           1184           27.25   
39997        22505             25152           1043           31.01   
39998        20013              2703            483           97.85   
39999        50137             55575           1175           34.63   

       Attack Type Encoded  
0                        2  
1                        2  
2                        0  
3                        2  
4 

### Нормализация данных

In [10]:
X = dataSet.drop('Attack Type Encoded', axis=1) 
y = dataSet['Attack Type Encoded']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Обучение и оценка качества на линейном ядре


In [11]:
param_grid = {'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
              'C': [0.001, 0.01, 0.1, 1, 10],
              'degree': [1, 2, 3, 4, 5, 6, 7]}  

svm_grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

svm_grid.fit(X_test, y_test)

best_params = svm_grid.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']
best_degree = best_params['degree']

print(f"Лучшие параметры: Ядро = {best_kernel}, C = {best_C}, Degree = {best_degree}")

Лучшие параметры: Ядро = poly, C = 0.1, Degree = 6


### Повторное обучение на лучших параметрах

In [13]:
svm_model = SVC(kernel='poly', degree=6, C=0.1) 

# Обучение модели
svm_model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = svm_model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)

print(f'Точность модели: {accuracy}')
print(report)

# Кросс-валидация
stratified_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm_model, X_train, y_train, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')



Точность модели: 0.3301875
              precision    recall  f1-score   support

           0       0.33      0.90      0.48      5306
           1       0.31      0.02      0.04      5416
           2       0.33      0.07      0.12      5278

    accuracy                           0.33     16000
   macro avg       0.32      0.33      0.21     16000
weighted avg       0.32      0.33      0.21     16000

Средняя точность перекрестной проверки: 0.33487500000000003
